In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/Transform_BertModel

/content/gdrive/MyDrive/Transform_BertModel


In [ ]:
!pip install nltk
!pip install hazm

!pip install  -q parsivar
!pip install pyLDAvis==3.2.2

!pip install convert_numbers

!pip -q install "gensim==3.8.1"
!pip install -q python-bidi
!pip install -q arabic_reshaper

############################
!pip install -qU sentence-transformers
!pip install -qU wikipedia-api
!pip install -qU hazm
!pip install -qU clean-text[gpl]
!pip install -qU emoji
!pip install -q bertopic==0.3.2

!apt-get install xz-utils -qy

!pip uninstall emoji
!pip install emoji==1.7

!pip install joblib==1.1.0

!pip install clean-text[gpl]
#########################
!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 5.1 MB/s 
     |████████████████████████████████| 233 kB 56.0 MB/s 
     |████████████████████████████████| 1.4 MB 54.7 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394488 sha256=2b74b1880d87fe2a4329c69444f51899cdd26786366a70354ae8a6f2983475d6
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153941 sha256=cccb51a142da5839d9777fc2ed26228401a852ff0d0b84088bbf69717c1fe251
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 

In [ ]:
# from preprocessing import cleaning
from utils import num_lines_in_file
# from IPython import displayy

import nltk
import wikipediaapi

import requests
from tqdm import tqdm
import json

from typing import List
from hazm import *

import re
import numpy as np
import pandas as pd
from numpy import array

from wordcloud import WordCloud
from PIL import Image, ImageFont, ImageDraw
from bidi.algorithm import get_display
import arabic_reshaper
import matplotlib.pyplot as plt

import time

import torch
from sentence_transformers import models, SentenceTransformer, util
from bertopic import BERTopic
######################################
def normal(text):
    #normalize the text
    normalizer = Normalizer()
    text=str(text)
    text = normalizer.character_refinement(text)
    text = normalizer.punctuation_spacing(text)
    text = normalizer.affix_spacing(text)
    text = normalizer.normalize(text)
    return text

def remove_stopwords(text,my_stopwords):

    text=str(text)
    filtered_tokens = [token for token in word_tokenize(text) if token not in my_stopwords]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

def remove_emoji(text): 
    emoji_pattern = re.compile("["
                u"\U0001F600-\U0001F64F"  # emoticons
                u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                u"\U0001F680-\U0001F6FF"  # transport & map symbols
                u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                u"\U00002702-\U000027B0"
                u"\U000024C2-\U0001F251"
                u"\U0001f926-\U0001f937"
                u'\U00010000-\U0010ffff'
                u"\u200d"
                u"\u200c"
                u'\u200f'
                u'\u200e'
                u"\u2640-\u2642"
                u"\u2600-\u2B55"
                u"\u23cf"
                u"\u23e9"
                u"\u231a"
                u"\u3030"
                u"\ufe0f"
                u"\u202b"

                u"\U000E006E|" \
                u"\U000E007F|" \
                u"\U000E0073|" \
                u"\U000E0063|" \
                u"\U000E0074|" \
                u"\U000E0077|" \
                u"\U000E006C"

    "]+", flags=re.UNICODE)
    
    return emoji_pattern.sub(r' ', text)

def remove_link(text): 
    return re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', str(text))

def remove_tabs(text): 
    return re.sub(r'[\n\r\t]', '', str(text))

def remove_email(text): 
    return re.sub(r'\S+@\S+', '', str(text))

def remove_englishword(text): 
    return re.sub(r'[A-Za-z0-9]+', ' ', str(text))

def remove_chars(text): 
    # return re.sub(r'\.(?!\d)', '', str(text))
    # return  re.sub(r'[$+&+;+]|[><!+،:,\(\).+،+٬+,+٬]|[-+]|[…]|[\[\]»«//]|[\\]|[#+]|[_+]|[٪+]|[%]|[*+]|[؟+]|[?+]|[""]|@|' '|[٠|١|۴|۵|۶|۷|۸|۲|۳|۰|٤|٣|٢|۹|٨|٧|٦|۱|٥]', '', str(text))
    return  re.sub(r'[$+&+;+]|[><!+،:,\(\).+،+٬+,+]|[-+]|[…]|[\[\]»«//]|[\\]|[#+]|[_+]|[٪+]|[%]|[*+]|[؟+]|[?+]|[""]|@|[٠|١|۴|۵|۶|۷|۸|۲|۳|۰|٤|٣|٢|۹|٨|٧|٦|۱|٥]', ' ', str(text))

def remove_extraspaces(text):
    return re.sub(r' +', ' ', text)

def remove_extranewlines(text):
    return re.sub(r'\n\n+', '\n\n', text)

def handle_clear_more_triple_chars(text):
    # remove any char that appear more than 2 times Continuously except whitespaces, tabs and newlines
    doc_string=re.sub("(.)\\1{2,}", "\\1", text)
    return doc_string

def preprocess(text,my_stopwords):
    text = normal(text)
    text = remove_stopwords(text,my_stopwords)
    text = remove_chars(text)
    text = remove_emoji(text)
    text = remove_link(text)
    text = remove_tabs(text) 
    text = remove_email(text) 
    text = remove_englishword(text) 
    text = remove_extraspaces(text) 
    text = remove_extranewlines(text) 
    text = handle_clear_more_triple_chars(text)
    return text

######################################
def create_dict(keys, values):
    topics = {}
    for key in set(keys):
        topics[key] = ([values[i] for i, value in enumerate(keys) if value == key])
    
    new_topics = {}
    for key in topics.keys():
        val = sum(topics[key])
        if val and len(key)>1:
            new_topics[key] = val  * 10000 
        else: continue
    return new_topics

def remove_unusful_topics(topics):
    return {x: y for x, y in topics.items() if not x.endswith("های") and not x.endswith("هایی")}

def remove_nested_keywords(topics):

    keys = list(topics.keys())
    values = list(topics.values())
    for i, sub_item in enumerate(keys):
        for j, item in enumerate(keys):
            if i==j:
                continue
            elif sub_item in item:
                keys[i] = "-1-1+1"
                values[i] = "-1-1+1"

    keys = list(filter(lambda a: a != "-1-1+1", keys))
    values = list(filter(lambda a: a != "-1-1+1", values))

    new = {}
    for i in range(len(keys)):
        try:
            new_key = keys[i]
            if len(new_key.split())>1 and new_key.split().count("_") > 1 and not new_key.split()[-1]!="_" and not new_key.split()[0] != "_":
                new_key = '\u200c'.join(new_key.split())
            if len(new_key) > 3:
                new[new_key] = values[i]
        except: continue
    return new



def wordcloud(topics):
        image_address = "/content/gdrive/MyDrive/2_Train_Bertopic/Insta.png"
        mask = np.array(Image.open("/content/gdrive/MyDrive/Transform_BertModel/mask-instagram.png"))
        wordcloud = WordCloud(max_font_size=80, background_color="white", font_path="/content/gdrive/MyDrive/Transform_BertModel/Vazir-Bold.ttf",
                              max_words=80, mask=mask, 
                              margin=10, height=800, width=800, colormap="Dark2", prefer_horizontal=1)
        new_topics = {}
        # box_size = draw.textsize(word, font=transposed_font)
        for x, y in topics.items():
            try:
                new_topics[get_display(arabic_reshaper.reshape(x))] = y
            except: continue
        print(new_topics)
        topics = new_topics
        wordcloud.generate_from_frequencies(topics)
        wordcloud.to_file(image_address)
        image = Image.open(image_address)
        image.thumbnail((800, 800), Image.ANTIALIAS)
        image = image.save(image_address, 'png', quality=100)
        # plt.imshow(wordcloud)
        plt.show()
        
        return topics
######################################
def my_data(voa_corpus_path_txt,stop_txt):

  stopwords = []
  with open(stop_txt, 'r') as fp:
      for line in fp:
          x = line[:-1]
          stopwords.append(x)

  print("************************")
  print(len(stopwords))
  print("************************")
  
  data = []
  counter = 0
  with open(voa_corpus_path_txt, encoding='utf-8') as f:
      _data = {'text': ''}
      texts = []
      for line in tqdm(f, total=num_lines_in_file(voa_corpus_path_txt)):
          line = line.strip()
          if counter>0 :
              _data['text'] = '\n'.join(texts).strip()
              data.append(_data) ###

              _data = {'text': ''}
              texts = []
          texts.append(line)
          counter += 1
  return data,stopwords
#######################################
def load_st_model(model_name_or_path):
  word_embedding_model = models.Transformer(model_name_or_path)
  pooling_model = models.Pooling(
      word_embedding_model.get_word_embedding_dimension(),
      pooling_mode_mean_tokens=True,
      pooling_mode_cls_token=False,
      pooling_mode_max_tokens=False)
  
  model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
  return model
########################################
def my_run(file_name_text,stop_txt):

  data,my_stopwords= my_data(file_name_text,stop_txt)

  print("strat ...")
  start_time = time.time()

  df = pd.DataFrame(data)
  df = df[['text']]
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  print(f'We have #{len(df)} news!')
  print("*********************")
  print(len(my_stopwords))
  df['text'] = df['text'].apply(preprocess,args = [my_stopwords])
  df['text'] = df['text'].dropna()
  df['text']= pd.DataFrame(df[df['text'].map(len) > 2]["text"])
  df = pd.DataFrame(df['text'].dropna()).reset_index()

  documents = df['text'].tolist() #.sample(frac=0.2).values.tolist() # sample is 0.2 times data
  print(f'We have #{len(documents)} news!')


  # _model = load_st_model('m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens')
  # _model.save('/content/gdrive/MyDrive/TopicModeling/bert-fa-base-uncased-wikinli-mean-tokens/')

  # model = BERTopic(
  #     '/content/gdrive/MyDrive/TopicModeling/bert-fa-base-uncased-wikinli-mean-tokens/',
  #     stop_words=my_stopwords,
  #     verbose=True)  

  my_model = BERTopic.load("/content/gdrive/MyDrive/2_Train_Bertopic/Insta_model")  
  new_topics, new_probabilities = my_model.transform(documents)
  ##################################
  end_pre = time.time() - start_time
  print("preprocessing finished: " + str(end_pre) + "\n")

  print(my_model.get_topics())
  print(len(my_model.get_topics().keys()))
  print("###########################")
  # print(new_topics)
  print("###########################")
  #############################
  result = {}
  for d in (my_model.get_topics()).values():
    result.update(d)
  #############################
  # topics= create_dict(keys, values)
  topics= remove_unusful_topics(result)
  topics= remove_nested_keywords(result)
  wordcloud(topics)

  return topics

topics= my_run('/content/gdrive/MyDrive/Transform_BertModel/Insta_ALL_comment.txt','/content/gdrive/MyDrive/Transform_BertModel/my_stopwords.txt')

************************
13754
************************


20968it [00:00, 461965.22it/s]           


strat ...
We have #14955 news!
*********************
13754
We have #12911 news!
preprocessing finished: 511.70487236976624

{-1: [('تخفیف', 0.05687490653087146), ('قیمت', 0.043069288809713994), ('خانومی', 0.04208101893251404), ('محصولات', 0.03715668105988979), ('محصول', 0.03652482841220852), ('کرم', 0.03275705027163665), ('سایت', 0.031879814280374084), ('مو', 0.027169034080872085), ('عالیه', 0.02199643977049117), ('عکس', 0.019524435944509973), ('رنگ', 0.01878273247258442), ('آرایش', 0.018730875426554137), ('سفارش', 0.01665118219934806), ('لافارر', 0.01650825116876028), ('هدیه', 0.016477624848786863), ('بن', 0.016359759118857283), ('خوبه', 0.01609157964804828), ('پاک', 0.01606444901721265), ('ضدآفتاب', 0.01604353885440971), ('کد', 0.015547178195772735)], 0: [('همشه', 9.46583494017472), ('ییمت', 0.0), ('داگلاسو', 0.0), ('داروخانه', 0.0), ('داروخونه', 0.0), ('داروساز', 0.0), ('داروهای', 0.0), ('داروی', 0.0), ('دارویی', 0.0), ('داریدازش', 0.0), ('داریدبدون', 0.0), ('داریمش', 0.0), ('داریند

In [ ]:
len(my_stopwords)

In [ ]:
import pandas as pd


def sum(x, y, z):
    return x + y + z


df = pd.DataFrame({'A': [1, 2], 'B': [10, 20]})

df1 = df.apply(sum, args=(222, 111))
print(df1)

     A    B
0  334  343
1  335  353


In [ ]:
df

,A,B
0,1,10
1,2,20
